In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from ebmdatalab import bq

In [26]:
#grab table names

sql = """
SELECT
 staging.*
FROM
 staging_measures.INFORMATION_SCHEMA.TABLES as staging
 join measures.INFORMATION_SCHEMA.TABLES as live 
on staging.table_name=live.table_name
 where
 staging.table_name like 'ccg%'
 order by staging.creation_time DESC

"""
table_df = bq.cached_read(sql, csv_path='staging_df.csv', use_cache=False)

In [27]:
table_df.head(200)

,table_catalog,table_schema,table_name,table_type,is_insertable_into,is_typed,creation_time
0,ebmdatalab,staging_measures,ccg_data_trimethoprim,BASE TABLE,YES,NO,2019-04-05 07:57:14.277
1,ebmdatalab,staging_measures,ccg_data_tramadol,BASE TABLE,YES,NO,2019-04-05 07:56:25.194
2,ebmdatalab,staging_measures,ccg_data_test_strip,BASE TABLE,YES,NO,2019-04-05 07:55:37.821
3,ebmdatalab,staging_measures,ccg_data_tamoxifen,BASE TABLE,YES,NO,2019-04-05 07:54:22.309
4,ebmdatalab,staging_measures,ccg_data_solublepara,BASE TABLE,YES,NO,2019-04-05 07:52:47.400
5,ebmdatalab,staging_measures,ccg_data_silver,BASE TABLE,YES,NO,2019-04-05 07:51:50.460
6,ebmdatalab,staging_measures,ccg_data_sildenafil,BASE TABLE,YES,NO,2019-04-05 07:51:05.092
7,ebmdatalab,staging_measures,ccg_data_saba,BASE TABLE,YES,NO,2019-04-05 07:50:16.265
8,ebmdatalab,staging_measures,ccg_data_pregabalinmg,BASE TABLE,YES,NO,2019-04-05 07:49:06.805
9,ebmdatalab,staging_measures,ccg_data_pregabalin,BASE TABLE,YES,NO,2019-04-05 07:48:14.899


In [22]:
#obtain AAF and EHF data from BQ, using measures.cpma_products as reference list
staging_df=pd.DataFrame()
for table_name in table_df['table_name']:
    
    sql = """

    SELECT '{}' as table_name, sum(a.numerator)-sum(b.numerator) AS num_dif, sum(a.denominator)-sum(b.denominator) AS den_diff
        FROM 
      `ebmdatalab.staging_measures.{}` AS a
    INNER JOIN
      `ebmdatalab.measures.{}` AS b
    ON
      a.month = b.month
      AND a.pct_id = b.pct_id
    --WHERE round(a.numerator,2) != round(b.numerator,2)
    --or round(a.denominator,2) != round(b.denominator,2)
    group by table_name

    
    """
    sql = sql.format(table_name, table_name, table_name)
    staging_df = pd.concat([staging_df, bq.cached_read(sql, '{}_staging_df.csv'.format(table_name), use_cache=False)])
    staging_df.to_csv("stagingtest4.csv")



In [17]:
table_df.head(200)

,table_name
0,ccg_data_statinintensity
1,ccg_data_diltiazem
2,ccg_data_nebivolol
3,ccg_data_lpneedles
4,ccg_data_ktt9_antibiotics
5,ccg_data_lpbathshoweremollients
6,ccg_data_sildenafil
7,ccg_data_lpcoprox
8,ccg_data_lpaliskiren
9,ccg_data_opioidspercent
